## How to make Gst.Buffer writable

Explained: http://lifestyletransfer.com/how-to-make-gstreamer-buffer-writable-in-python/

In [1]:
import gi
gi.require_version('Gst', '1.0')
gi.require_version('GstBase', '1.0')
from gi.repository import Gst, GObject, GstBase

In [2]:
Gst.init(None)

[]

### Problem

In [3]:
buffer = Gst.Buffer.new_wrapped(b"lifestyletransfer")

In [4]:
ret, map_info = buffer.map(Gst.MapFlags.READ | Gst.MapFlags.WRITE)

In [5]:
map_info.data

b'lifestyletransfer'

In [6]:
map_info.data[0] = "L"

TypeError: 'bytes' object does not support item assignment

### Solution

In [7]:
from ctypes import *
from contextlib import contextmanager

In [8]:
GST_PADDING = 4

In [9]:
libgst = CDLL("libgstreamer-1.0.so.0")

In [ ]:
class GstMapInfo(Structure):
    _fields_ = [("memory", c_void_p),        # GstMemory *memory
                ("flags", c_int),            # GstMapFlags flags
                ("data", POINTER(c_byte)),   # guint8 *data
                ("size", c_size_t),          # gsize size
                ("maxsize", c_size_t),       # gsize maxsize
                ("user_data", c_void_p * 4), # gpointer user_data[4]
                ("_gst_reserved", c_void_p * GST_PADDING)]

In [ ]:
GST_MAP_INFO_POINTER = POINTER(GstMapInfo)

In [ ]:
# gst_buffer_map
libgst.gst_buffer_map.argtypes = [c_void_p, GST_MAP_INFO_POINTER, c_int]
libgst.gst_buffer_map.restype = c_bool

# gst_buffer_unmap
libgst.gst_buffer_unmap.argtypes = [c_void_p, GST_MAP_INFO_POINTER]
libgst.gst_buffer_unmap.restype = None

# gst_mini_object_is_writable
libgst.gst_mini_object_is_writable.argtypes = [c_void_p]
libgst.gst_mini_object_is_writable.restype = c_bool

In [ ]:
@contextmanager
def map_gst_buffer(pbuffer, flags):
    if pbuffer is None:
        raiseTypeError("Cannot pass NULL to _map_gst_buffer")

    ptr = hash(pbuffer)
    if flags & Gst.MapFlags.WRITE and libgst.gst_mini_object_is_writable(ptr) == 0:
        raiseValueError("Writable array requested but buffer is not writeable")

    mapping = GstMapInfo()
    success = libgst.gst_buffer_map(ptr, mapping, flags)

    if not success:
        raiseRuntimeError("Couldn't map buffer")

    try:
        yield cast(mapping.data, POINTER(c_byte * mapping.size)).contents
    finally:
        libgst.gst_buffer_unmap(ptr, mapping)

In [ ]:
buffer = Gst.Buffer.new_wrapped(b"lifestyletransfer")

In [ ]:
with map_gst_buffer(buffer, Gst.MapFlags.READ | Gst.MapFlags.WRITE) as mapped:
    mapped[0] = ord('L')
    mapped[4] = ord('S')
    mapped[9] = ord('T')

In [ ]:
ret, map_info = buffer.map(Gst.MapFlags.READ)
print(map_info.data)